Test to see how well the Levenshtein distance can be used as a predictor for relevant datasets:

In [21]:
# Import required modules
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.SubsMat.MatrixInfo import blosum62
from Levenshtein import distance as levenshtein_distance
from scipy.stats import spearmanr

# Create a new blosum matrix that has both directions. 
SYMMETRIC_BLOSUM = {}
for (aa1, aa2), score in blosum62.items():
    SYMMETRIC_BLOSUM[(aa1, aa2)] = score
    SYMMETRIC_BLOSUM[(aa2, aa1)] = score

# General Functions

In [22]:
def evaluate_task(task_filename, reference_seq):
    
    # Load the task file and filter to just the test data
    task_df = pd.read_csv(task_filename)
    task_df = task_df.loc[task_df.set == "test"].copy()

    # Create output arrays
    refseq_len = len(reference_seq)
    all_seqs = task_df.sequence.tolist()
    levenshteins = np.empty(len(all_seqs))
    blosum_scores = np.empty(len(all_seqs))
    
    # Calculate levenshtein distance between each sequence and the reference
    levenshteins = np.array([levenshtein_distance(reference_seq, new_seq) 
                             for new_seq in task_df.sequence.values])
    
    # Calculate scores for each sequence    
    calculate_blosum = True
    for i, new_seq in enumerate(all_seqs):

        # Score by levenshtein
        levenshteins[i] = levenshtein_distance(reference_seq, new_seq)
        
        # Continue to calculate blosum unless the data is not aligned
        if calculate_blosum:
            
            # Make sure the reference sequence and this sequence align
            seqs_aligned = len(new_seq) == refseq_len
            if not seqs_aligned:
                calculate_blosum = False
                blosum_scores = None
                continue
            
            # Calculate blosum scores
            blosum_scores[i] = sum(SYMMETRIC_BLOSUM[(aa1, aa2)] for 
                                   aa1, aa2 in zip(reference_seq, new_seq))

    # Now get spearman rho and record. Negative levenshtein because we
    # expect a smaller distance to be correlated to larger fitness.
    l_rho, _ = spearmanr(-levenshteins, task_df.target.values)
    if blosum_scores is not None:
        b_rho, _ = spearmanr(blosum_scores, task_df.target.values)
    else:
        b_rho = None
    
    return l_rho, b_rho

def evaluate_tasks(refseq_fileloc, taskfolder, task_to_file_dict):
    
    # Get the reference sequence
    reference_seq = str(next(SeqIO.parse(refseq_fileloc, "fasta")).seq)

    # Loop over each task
    results = [["Task", "Levenshtein Rho", "BLOSUM62 Rho"]]
    for taskname, taskfile in task_to_file_dict.items():
        rhos = evaluate_task(os.path.join(taskfolder, taskfile), 
                            reference_seq)
        results.append([taskname, *rhos])
        
    return results

# AAV

In [23]:
def levenshtein_to_fitness_aav():

    # Define the different aav inputs
    aav_refseq_file = "./data/FLIP/aav/P03135.fasta"
    aav_taskfolder = "./data/FLIP/aav/splits"
    aav_task_to_file = {
        "two_vs_many": "two_vs_many.csv",
        "seven_vs_many": "seven_vs_many.csv",
        "low_vs_high": "low_vs_high.csv"
    }

    return evaluate_tasks(aav_refseq_file,
                          aav_taskfolder,
                          aav_task_to_file)

levenshtein_to_fitness_aav()

[['Task', 'Levenshtein Rho', 'BLOSUM62 Rho'],
 ['two_vs_many', 0.5776311422394775, None],
 ['seven_vs_many', 0.550377598162819, None],
 ['low_vs_high', 0.2506663869079836, None]]

# GB1

In [24]:
def levenshtein_to_fitness_gb1():
    
    # Define the inputs
    refseq_file = "./data/FLIP/gb1/5LDE_1.fasta"
    taskfolder = "./data/FLIP/gb1/splits"
    task_to_file = {
        "two_vs_rest": "two_vs_rest.csv",
        "three_vs_rest": "three_vs_rest.csv",
        "low_vs_high": "low_vs_high.csv"
    }
    
    return evaluate_tasks(refseq_file,
                          taskfolder,
                          task_to_file)

levenshtein_to_fitness_gb1()

[['Task', 'Levenshtein Rho', 'BLOSUM62 Rho'],
 ['two_vs_rest', 0.15567498369768723, 0.12831594743289912],
 ['three_vs_rest', -0.06920804785431008, 0.00486406694071143],
 ['low_vs_high', -0.10779246349106573, -0.1271557890096999]]

# Fluorescence

In [ ]:
# # write string to .fasta file
# def write_fasta(file_name, sequence, description=""):
#     with open(file_name, "w") as fasta_file:
#         fasta_file.write(f">{description}\n")
#         # Split the sequence into lines of 80 characters each
#         for i in range(0, len(sequence), 80):
#             fasta_file.write(sequence[i:i + 80] + "\n")
# 
# # Example usage:
# fasta_file_name = "../data/PEER/fluorescence/wt_sequence.fasta"
# sequence_description = "Wild type Sequence"
# # sequence_data = "ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG"
# 
# write_fasta(fasta_file_name, wt_sequence, sequence_description)

In [19]:
def evaluate_task_fluorescence(task_filename, reference_seq):
    
    # Load the task file and filter to just the test data
    task_df = pd.read_csv(task_filename)
    task_df = task_df.loc[task_df.set == "test"].copy()

    # Create output arrays
    refseq_len = len(reference_seq)
    # all_seqs = task_df.sequence.tolist()
    all_seqs = task_df.primary.tolist()
    levenshteins = np.empty(len(all_seqs))
    blosum_scores = np.empty(len(all_seqs))
    
    # Calculate levenshtein distance between each sequence and the reference
    # levenshteins = np.array([levenshtein_distance(reference_seq, new_seq) 
    #                          for new_seq in task_df.sequence.values])
    levenshteins = np.array([levenshtein_distance(reference_seq, new_seq) 
                             for new_seq in task_df.primary.values])
    
    # Calculate scores for each sequence    
    calculate_blosum = True
    for i, new_seq in enumerate(all_seqs):

        # Score by levenshtein
        levenshteins[i] = levenshtein_distance(reference_seq, new_seq)
        
        # Continue to calculate blosum unless the data is not aligned
        if calculate_blosum:
            
            # Make sure the reference sequence and this sequence align
            seqs_aligned = len(new_seq) == refseq_len
            if not seqs_aligned:
                calculate_blosum = False
                blosum_scores = None
                continue
            
            # Calculate blosum scores
            blosum_scores[i] = sum(SYMMETRIC_BLOSUM[(aa1, aa2)] for 
                                   aa1, aa2 in zip(reference_seq, new_seq))

    # Now get spearman rho and record. Negative levenshtein because we
    # expect a smaller distance to be correlated to larger fitness.
    # l_rho, _ = spearmanr(-levenshteins, task_df.target.values)
    l_rho, _ = spearmanr(-levenshteins, task_df.log_fluorescence.values)
    if blosum_scores is not None:
        b_rho, _ = spearmanr(blosum_scores, task_df.target.values)
    else:
        b_rho = None
    
    return l_rho, b_rho

def evaluate_tasks_fluorescence(refseq_fileloc, taskfolder, task_to_file_dict):
    
    # Get the reference sequence
    reference_seq = str(next(SeqIO.parse(refseq_fileloc, "fasta")).seq)

    # Loop over each task
    results = [["Task", "Levenshtein Rho", "BLOSUM62 Rho"]]
    for taskname, taskfile in task_to_file_dict.items():
        rhos = evaluate_task_fluorescence(os.path.join(taskfolder, taskfile), 
                            reference_seq)
        results.append([taskname, *rhos])
        
    return results

In [20]:
def levenshtein_to_fitness_fluorescence():
    
    # Define the inputs
    refseq_file = "./data/PEER/fluorescence/wt_sequence.fasta"
    taskfolder = "./data/PEER/fluorescence/splits"
    task_to_file = {
        "two_vs_rest": "two_vs_many.csv",
        "three_vs_rest": "seven_vs_many.csv",
        "low_vs_high": "low_vs_high.csv"
    }
    
    return evaluate_tasks_fluorescence(refseq_file,
                          taskfolder,
                          task_to_file)

levenshtein_to_fitness_fluorescence()

[['Task', 'Levenshtein Rho', 'BLOSUM62 Rho'],
 ['two_vs_rest', 0.46596018330742606, None],
 ['three_vs_rest', 0.05396761248845623, None],
 ['low_vs_high', 0.011234704728670488, None]]